In [ ]:
import json

# Đọc file JSON
def load_data_from_json(path):
    with open(path, 'r') as f:
        data = json.load(f)

    return data

def save_data_to_json(data, path):
    with open(path, 'w', encoding='utf-8') as fp:
        json.dump(data, fp, ensure_ascii=False, indent=4)

In [ ]:
data = load_data_from_json("final_data_new.json")
context_data = load_data_from_json("merged_data_file.json")

In [ ]:
# !pip install phonlp

In [ ]:
import pandas as pd

# Gộp dữ liệu
for item in data:
    context_id = item['metadata']['context_id']
    item['context'] = context_data[context_id]['context']

# Chuyển đổi thành DataFrame
df = pd.DataFrame(data)
# Xáo trộn dữ liệu
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Trích xuất các trường cần thiết
claims = df['claim'].values
contexts = df['context'].values
labels = df['metadata'].apply(lambda x: 1 if x['label'] == 'HỖ TRỢ' else 0).values

# Tokenization và padding
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(np.concatenate((claims, contexts)))

X_claims = pad_sequences(tokenizer.texts_to_sequences(claims), maxlen=100)
X_contexts = pad_sequences(tokenizer.texts_to_sequences(contexts), maxlen=300)

# Gộp claim và context
X = np.concatenate((X_claims, X_contexts), axis=1)
y = to_categorical(labels, num_classes=2)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from torch import nn
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Giả sử df là DataFrame của bạn
X = df[['claim', 'context']]
y = df['metadata'].apply(lambda x: 1 if x['label'] == 'HỖ TRỢ' else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
phobert = AutoModel.from_pretrained(model_name)

In [ ]:
class PhoBERTClassifier(nn.Module):
    def __init__(self, phobert):
        super(PhoBERTClassifier, self).__init__()
        self.phobert = phobert
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, 2)  # 768 là kích thước đầu ra của PhoBERT, 2 là số lớp

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

model = PhoBERTClassifier(phobert)

In [ ]:
def prepare_data(texts, labels, tokenizer, max_length=512):
    encodings = tokenizer(texts['claim'].tolist(), texts['context'].tolist(),
                          truncation=True, padding='max_length', max_length=max_length, return_tensors="pt")
    dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels.tolist()))
    return dataset

max_length = 256  # hoặc một giá trị phù hợp với dữ liệu của bạn
train_dataset = prepare_data(X_train, y_train, tokenizer, max_length=max_length)
test_dataset = prepare_data(X_test, y_test, tokenizer, max_length=max_length)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
      input_ids, attention_mask, labels = [b.to(device) for b in batch]
      optimizer.zero_grad()
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

    # Đánh giá trên tập kiểm tra
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            true_labels.extend(labels.cpu().tolist())

    accuracy = accuracy_score(true_labels, predictions)
    print(f"Epoch {epoch+1}/{num_epochs}, Test Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(true_labels, predictions))